# What is this about?

This is a demo file to showcase how the CONCERT modules can be visualized using TUM's timor python.

If you have any questions, contact:
- [Jonathan](jonathan.kuelz@tum.de), or
- [Matthias](matthias.mayer@tum.de)

In [1]:
# The meshcat visualizer does sometimes not terminate properly. You can kill all local processes with "meshcat" in there name by runnning this cell.
! pkill -f meshcat

In [2]:
%load_ext autoreload
%autoreload 2
import json
import meshcat
import os
from pathlib import Path

import numpy as np

# If you get a file not found error at some point, enter your file locations here
concert_folder = Path(os.path.abspath("")).parent
jsons = concert_folder.joinpath('json/concert/')

**Timor**? [Timor python](https://gitlab.lrz.de/tum-cps/timor-python) is a modular robot toolbox designed @ TU Munich. At the moment, you need to checkout the branch `dev/new_concert_modules` in order to make the following code work. We recommend local, editeable pip install.

In [3]:
from pinocchio.visualize.meshcat_visualizer import MeshcatVisualizer
from timor import AtomicModule, ModuleAssembly, ModulesDB, Transformation

In [4]:
colors = {}  # A little hack because Timor does not support body colors natively
modules = set()  # We are gonna parse them one by one
for f in jsons.iterdir():
    m, c = AtomicModule.from_concert_specification(json.loads(f.open('rb').read()), concert_folder.parent)
    modules.add(m)
    colors[m.id] = c
db = ModulesDB(modules)

## Let's see the modules
Use the cell below to visualize modules interactively. This will open an integrated _meshcat_ visualizer. There you can:
- Move and pan the camera
- Manipulate to scene by opening the dropdown menu on the top right. Especially you can
    - Manipulate lighting and appearance
    - Show or hide collision (and visual) geometries
    - Under "meshcat", hide and show the various coordinate systems that timor defines. They indicate connector placement, center of mass, and more...'

In [5]:
%%capture
viz = MeshcatVisualizer()
viz.initViewer()
# If you prefer to have the visualizer in a distinct tab, you can can viz.viewer.open()

In [6]:
for attribute in ('visuals', 'collisions', 'meshcat'):
    viz.viewer.window.send(meshcat.commands.Delete(attribute))
id_or_name = '18'  # Enter here
if id_or_name in db.by_id:
    module = db.by_id[id_or_name]
elif id_or_name in db.by_name:
    module = db.by_name[id_or_name]
else:
    raise ValueError("Unknown module ID or name {}".format(id_or_name))

module.debug_visualization(viz)
viz.viewer.jupyter_cell(height=800)

# Visualize the CONCERT robot

Now, we are going to visualize the robot in a typical arrangement:

In [7]:
%%capture
robot_vis = MeshcatVisualizer()  # We are using a new visualizer so the module above will still be visible
robot_vis.initViewer()
# If you prefer to have the visualizer in a distinct tab, you can can viz.viewer.open()

In [13]:
assembly.robot.random_configuration()

[autoreload of timor.Geometry failed: Traceback (most recent call last):
  File "/home/jonathan/anaconda3/envs/timor/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/jonathan/anaconda3/envs/timor/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/home/jonathan/anaconda3/envs/timor/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/home/jonathan/anaconda3/envs/timor/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/home/jonathan/anaconda3/envs/timor/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 303, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: can't apply this __setattr__ to HppfclEnumMeta object
]


array([-1.70857635e+00,  1.61091441e+00,  2.37536371e+00,  1.51399823e+00,
       -2.98419818e+00,  5.17112766e+09, -8.80936045e+09,  6.10427098e+08,
       -5.07497528e+09,  1.76965382e+00, -1.05002666e+00, -4.64573028e-01,
        2.67435599e+00, -2.14445286e+00])

In [16]:
from timor.utilities.visualization import color_visualization
modules_used = ('23', '14', '16', '14', '16', '14', '16', '17', '17', '18', '18', '51', '51', '51', '51')
# Connections have to be specified precisely, because the CONCERT definition would theoretically allow adding manipulator modules to the wheel mounting flanges
connections=(
    (0, '23_connector_5', 1, '14_input'),
    (1, '14_output', 2, '16_input'),
    (2, '16_output', 3, '14_input'),
    (3, '14_output', 4, '16_input'),
    (4, '16_output', 5, '14_input'),
    (5, '14_output', 6, '16_input'),
    (0, '23_connector_1', 9, '18_input'),  # front_left
    (0, '23_connector_2', 7, '17_input'),  # front_right
    (0, '23_connector_3', 8, '17_input'),  # rear_left
    (0, '23_connector_4', 10, '18_input'),  # rear_right
    (7, '17_output', 11, '51_input'), (8, '17_output', 12, '51_input'), (9, '18_output', 13, '51_input'), (10, '18_output', 14, '51_input'),  # wheels
)
assembly = ModuleAssembly(db, modules_used, connections, base_connector=(0, '23_connector_base'), base_placement=Transformation.from_translation((0, 0, .58)))
assembly.robot.update_configuration(np.array([0,  0,  0,  0, -2.9,  1.17, -1.8,  2.1, -.07497528,  1.7, -1.05, -0.46, 2.6, -2.1]))
assembly.robot.visualize(robot_vis)
color_visualization(robot_vis, assembly, colors)
robot_vis.viewer.jupyter_cell(height=1600)

Introducing 8 joints for the 4 wheel mounts + wheels is necessary for motion planning.

However, for robot optimization within timor, this introduces a level of complexity we don't want to deal with. Therefore, we introduce a simplified base that models the assembled concert robot without wheels, just as a moving and floating base.

In [ ]:
%%capture
simple_vis = MeshcatVisualizer()  # We are using a new visualizer so the module above will still be visible
simple_vis.initViewer()
# If you prefer to have the visualizer in a distinct tab, you can can viz.viewer.open()

In [ ]:
from timor.utilities.visualization import color_visualization
modules_used = ('23_planar', '14', '16', '14', '16', '14', '16')
connections=(
    (0, '23_connector_5', 1, '14_input'),
    (1, '14_output', 2, '16_input'),
    (2, '16_output', 3, '14_input'),
    (3, '14_output', 4, '16_input'),
    (4, '16_output', 5, '14_input'),
    (5, '14_output', 6, '16_input'),
)
assembly = ModuleAssembly(db, modules_used, connections, base_connector=(0, '23_connector_base'))
color_visualization(simple_vis, assembly, colors)
simple_vis.viewer.jupyter_cell(height=800)

# Let's have fun

From here on, you can use Timor and CONCERT modules however you like to. Here's a fun animation as a final demo. Note that Timor does not intend to simulate moving bases, therefore we will just exclude the wheels for now.

Quick reminder: This is only a visualization demo. We do not perform collision checking, nor do we make sure that there is any sense behind the trajectories. It's just to showcase that the modules have been fully integrated and can further be used for arbitrary tasks.

In [ ]:
%%capture
anim_vis = MeshcatVisualizer()  # We are using a new visualizer so the module above will still be visible
anim_vis.initViewer()
# If you prefer to have the visualizer in a distinct tab, you can can viz.viewer.open()

In [ ]:
from timor.utilities.visualization import animation
from timor.utilities.trajectory import Trajectory
random_configurations = np.asarray([assembly.robot.random_configuration() for _ in range(20)])
random_configurations[:, :3] = random_configurations[:, :3] / 4  # Limit the base movement a bit for the random animation
random_configurations = np.asarray([q for q in random_configurations if not assembly.robot.has_self_collision(q)])  # This will not guarantee a collision-free trajectory, but at least makes it more likely
trajectory = Trajectory.from_mstraj(np.vstack(random_configurations), dt=.02, t_acc=.1, qd_max=3.6)
color_visualization(anim_vis, assembly, colors)
animation(assembly.robot, trajectory.q, .1, anim_vis)
anim_vis.viewer.jupyter_cell(height=800)